In [4]:
import keras
import os
from deepsense import neptune
from keras_retinanet.bin.train import create_models
from keras_retinanet.callbacks.eval import Evaluate
from keras_retinanet.models.resnet import download_imagenet, resnet_retinanet as retinanet
from keras_retinanet.preprocessing.detgen import DetDataGenerator 
from keras_retinanet.utils.transform import random_transform_generator
from keras_retinanet.preprocessing.detgen import DetDataGenerator
from detdata import DetGen
from detdata.augmenters import crazy_augmenter

import warnings
warnings.filterwarnings("ignore")

ctx = neptune.Context()

detg= DetGen('/home/i008/malaria_data/dataset_train.mxrecords',
      '/home/i008/malaria_data/dataset_train.csv',
      '/home/i008/malaria_data/dataset_valid.mxindex', batch_size=4)

train_generator = DetDataGenerator(detg, augmenter=crazy_augmenter)
train_generator.image_max_side = 750
train_generator.image_min_side = 750




weights = download_imagenet('resnet50')

model_checkpoint = keras.callbacks.ModelCheckpoint('mod-{epoch:02d}_loss-{loss:.4f}.h5',
                                                   monitor='loss',
                                                   verbose=2,
                                                   save_best_only=False,
                                                   save_weights_only=False,
                                                   mode='auto',
                                                   period=1)

callbacks = []


class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        ctx.channel_send("loss", logs.get('loss'))

    def on_epoch_end(self, epoch, logs={}):
        ctx.channel_send("val_loss", logs.get('val_loss'))


callbacks.append(keras.callbacks.ReduceLROnPlateau(
    monitor='loss',
    factor=0.1,
    patience=2,
    verbose=1,
    mode='auto',
    epsilon=0.0001,
    cooldown=0,
    min_lr=0
))

# callbacks.append(LossHistory())
callbacks.append(model_checkpoint)

# model, training_model, prediction_model = create_models(
#     backbone_retinanet=retinanet,
#     backbone='resnet50',
#     num_classes=train_generator.num_classes(),
#     weights=weights,
#     multi_gpu=0,
#     freeze_backbone=True
# )

# training_model.fit_generator(
#     generator=train_generator,
#     steps_per_epoch=5000,
#     epochs=100,
#     verbose=1,
#     callbacks=callbacks,
# )

neptune: Executing in Offline Mode.


In [6]:
training_model.fit_generator(
    generator=train_generator,
    steps_per_epoch=5000,
    epochs=100,
    verbose=1,
    callbacks=callbacks,
)

Epoch 1/100
5000/5000 [==============================] - 2213s 443ms/step - loss: 2.3006 - regression_loss: 2.0442 - classification_loss: 0.2564

Epoch 00001: saving model to mod-01_loss-2.3006.h5
Epoch 2/100
5000/5000 [==============================] - 2273s 455ms/step - loss: 2.2346 - regression_loss: 1.9880 - classification_loss: 0.2466

Epoch 00002: saving model to mod-02_loss-2.2346.h5
Epoch 3/100
5000/5000 [==============================] - 2310s 462ms/step - loss: 2.2022 - regression_loss: 1.9605 - classification_loss: 0.2416

Epoch 00003: saving model to mod-03_loss-2.2022.h5
Epoch 4/100
5000/5000 [==============================] - 2167s 433ms/step - loss: 2.1729 - regression_loss: 1.9356 - classification_loss: 0.2373

Epoch 00004: saving model to mod-04_loss-2.1729.h5
Epoch 5/100
5000/5000 [==============================] - 2138s 428ms/step - loss: 2.1547 - regression_loss: 1.9198 - classification_loss: 0.2349

Epoch 00005: saving model to mod-05_loss-2.1547.h5
Epoch 6/100
500

KeyboardInterrupt: 